In [ ]:
%%capture
for module in ["tensorflow_addons"]:
    try:
        __import__(module)
    except ImportError:
        !pip install {module}

In [ ]:
import json
from collections import defaultdict
from ast import literal_eval

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, concatenate, SpatialDropout1D, GRU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
import os

import numpy as np
import pandas as pd

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_addons as tfa
# tf.compat.v1.enable_eager_execution()
from tensorflow.keras import backend as K
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow_hub as hub
from tensorflow.compat.v1.keras.callbacks import EarlyStopping

from tqdm.notebook import tqdm

from sklearn.metrics import classification_report

import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree

import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
DATA_DIR = os.path.join(BASE_DIR, "data")
MODEL_DIR = os.path.join(BASE_DIR, "model_output", "Experiments")
DATASET_LOC = os.path.join(DATA_DIR, "model_input", "dataset")
direct_parent = os.path.join(DATA_DIR, "GO_Category", "GO_DirectParents.tsv")

go_category = ["GO:0008150", "GO:0005575", "GO:0003674"]

In [ ]:
direct_data = pd.read_csv(direct_parent, delimiter="\t", names=['Child', 'Parent']).replace({"_": ":"}, regex=True).drop(0).reset_index(drop=True)
onto_digraph = nx.from_pandas_edgelist(direct_data, source='Child', target='Parent', create_using=nx.classes.digraph.DiGraph)
print("Number of nodes:", onto_digraph.number_of_nodes(), "\nNumber of edges:", onto_digraph.number_of_edges())

In [ ]:
subsumers = dict((i, list(set(np.array(dfs_tree(onto_digraph, i).edges()).flatten().tolist() + [i]) - set(["owl:Thing"]))) for i in onto_digraph.nodes())

In [ ]:
config = {
    "weight": 0.75,
    "learning_rate": 0.001,
    "epochs": 35,
    "batch_size": 64,
    "activation": 'softmax', 
    "rdropout": 0.3,
    "optimizer": 'adamw',
    "loss": 'sigfocalCE',
    "callbacks": [
                  {"early_stop": tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                patience = 10, 
                                                verbose = 1,
                                                restore_best_weights = True)},
    ],
    "learning_rate_func": 'cosinedecay',
    "max_len": 71,
    "max_char_len": 15,
    "min_sent_len": 3,
    "project": "Intelligent_OA",
    "extra_info": "ElMo, New Sim Score Scheme, inputs: Word(30D), POS(100D)",
    "dropout": 0.5,
    "name": "ELMO"
}

In [ ]:
train_data = json.load(open(os.path.join(DATASET_LOC, "train.json"), "r"))
train_data = [i for i in train_data if len(i['tokens']) >= config.get("min_sent_len")]

test_data = json.load(open(os.path.join(DATASET_LOC, "test.json"), "r"))
test_data = [i for i in test_data if len(i['tokens']) >= config.get("min_sent_len")]
input_data = train_data + test_data

all_data = {
    "tokens": [i['tokens'] for i in input_data],
    "tags": [i['iob_tags'] for i in input_data],
    "pos": [i['pos_tags'] for i in input_data],
}
assert (len(all_data['tokens']) == len(all_data['tags']) == len(all_data['pos']))

In [ ]:
words = ["PAD"] + sorted(set([j for i in all_data['tokens'] for j in i] + ["UNK", "O"]) - set(["PAD"]))
tags = ["PAD"] + sorted(set([j for i in all_data['tags'] for j in i] + ["UNK", "O"]) - set(["PAD"]))
chars = ["PAD"] + sorted(set([j for i in words for j in i] + ["UNK", "O"]) - set(["PAD"]))
pos = ["PAD"] + sorted(set([j for i in all_data['pos'] for j in i] + ["UNK", "O"]) - set(["PAD"]))

n_words, n_tags, n_chars, n_pos = len(words), len(tags), len(chars), len(pos)


print("\nNumber of Observations:{0}\nNumber of words:{1}\nNumber of tags:{2}\nNumber of characters: {3}\nNumber of pos: {4}".format(
    len(all_data['tokens']), n_words, n_tags, n_chars, n_pos))

In [ ]:
set([i.replace("B-", "").replace("I-", "") for i in tags]) - set(subsumers.keys())

In [ ]:
word2idx = dict((i, idx) for idx, i in enumerate(words))
idx2word = dict((v, k) for k, v in word2idx.items())

tag2idx = dict((i, idx) for idx, i in enumerate(tags))
idx2tag = dict((v, k) for k, v in tag2idx.items())

char2idx = dict((i, idx) for idx, i in enumerate(chars))
idx2char = dict((v, k) for k, v in char2idx.items())

pos2idx = dict((i, idx) for idx, i in enumerate(pos))
idx2pos = dict((v, k) for k, v in pos2idx.items())

In [ ]:
Y_tags = [[tag2idx.get(i) for i in sent] for sent in all_data['tags']]
Y_tags = pad_sequences(maxlen=config.get("max_len"), sequences=Y_tags, value=tag2idx.get("PAD"), padding='post', truncating='post', dtype='float16')
Y_tags = to_categorical(Y_tags, num_classes=n_tags, dtype='float16')

In [ ]:
def get_sim(term1, term2):
    if "GO" in term1 and "GO" in term2:
        term1 = term1.replace("B-", "").replace("I-", "")
        term2 = term2.replace("B-", "").replace("I-", "")
        t1 = set(subsumers.get(term1, term1))
        t2 = set(subsumers.get(term2, term2))
        if len(set.union(t1, t2)) > 0:
            simj=len(set.intersection(t1, t2))/len(set.union(t1, t2))
        else:
            simj = 0.0
    else:
        simj = 0.0
    return simj

In [ ]:
config

In [ ]:
sem_dist = dict([(i, to_categorical(i, num_classes=n_tags)) for i in range(n_tags)])
factor = 0
for i in range(n_tags):
    iob_i = None
    term_i = idx2tag.get(i)
    if "B-" in term_i or "I-" in term_i:
        iob_i = term_i[0]
    term_i = term_i.replace("B-", "").replace("I-", "")
    if "GO" in term_i:
        sem_scores = []
        for j in range(n_tags):
            iob_j = None
            term_j = idx2tag.get(j)
            if "B-" in term_j or "I-" in term_j:
                iob_j = term_j[0]
            term_j = term_j.replace("B-", "").replace("I-", "")
            score = config.get('weight') * get_sim(term_i, term_j)
            if iob_i != iob_j:
                score = factor * score
            sem_scores.append(score)
        sem_scores = np.array(sem_scores)
        # if config.get("weight"):
        #     unique_vals = np.unique(sem_scores)
        sem_scores[i] = 1
        sem_dist[i] = sem_scores

for i in range(n_tags):
    num_max = np.where(sem_dist[i] == 1)[0].size
    assert num_max == 1

for i in range(len(Y_tags)):
    for j in range(config.get("max_len")):
        k = np.where(Y_tags[i][j] == 1)[0][0]
        Y_tags[i][j] = sem_dist[k]

In [ ]:
for k, v in sem_dist.items():
    try:
        assert np.where(v == 1)[0].size == 1
    except:
        print(np.where(v == 1)[0])

In [ ]:
X_word = [(s, len(s)) for s in all_data['tokens']]
X_word = np.array([i[0][:config.get("max_len")] + [""]*(config.get("max_len")-i[1]) for i in X_word])

X_char_temp = []
for wds in all_data['tokens']:
    wds = wds[:config.get("max_len")] + ["PAD"]*(config.get("max_len") - len(wds))
    chrs = [list(word)[:config.get("max_char_len")] + ["PAD"]*(config.get("max_char_len")-len(word)) if word != "PAD" 
            else ["PAD"]*config.get("max_char_len") for word in wds]
    X_char_temp.append(np.array(chrs))
X_char_temp = np.array(X_char_temp)
X_char = np.vectorize(char2idx.get)(X_char_temp).astype('float16')
del X_char_temp

X_pos = [[pos2idx.get(w) for w in s] for s in all_data['pos']]
X_pos = pad_sequences(maxlen=config.get("max_len"), sequences=X_pos, value=pos2idx.get("PAD"), padding='post', truncating='post', dtype='float16')

In [ ]:
max_idx = 0
for i in range(len(X_word), 0, -1):
    if (i*0.7%config.get("batch_size") == 0) and (i*0.3%config.get("batch_size") == 0):
        max_idx = i
        break

combined = [(X_word[i], X_char[i], X_pos[i]) for i in range(max_idx)]
Y_tags = Y_tags[:max_idx]

X_tr, X_te, y_tr, y_te = train_test_split(combined, Y_tags, test_size=0.3, random_state=2022)

In [ ]:
input_train = []
for i in range(len(combined[0])):
    temp = []
    for j in range(len(X_tr)):
        temp.append(X_tr[j][i])
    if i == 0:
        input_train.append(np.array(temp, dtype='str'))
    else:
        input_train.append(np.array(temp, dtype='float16'))
# input_train = tuple(input_train)

input_test = []
for i in range(len(combined[0])):
    temp = []
    for j in range(len(X_te)):
        temp.append(X_te[j][i])
    if i == 0:
        input_test.append(np.array(temp, dtype='str'))
    else:
        input_test.append(np.array(temp, dtype='float16'))
# input_test = tuple(input_test)

In [ ]:
[i.shape for i in input_train], y_tr.shape, [i.shape for i in input_test], y_te.shape

In [ ]:
from sklearn.metrics import f1_score

def go_val_acc(y_true, y_pred):
    tmp = tf.math.argmax(y_pred[0], axis=-1)
    tmp_g = tf.math.argmax(y_true[0], axis=-1)
    #mask 0
    tmp_m = tf.math.equal(tmp_g, tf.constant(0, dtype=tf.int64))
    tmp_g_m = tf.math.equal(tmp, tf.constant(0, dtype=tf.int64))
    
    pr = tmp[tf.math.logical_not(tf.math.logical_and(tmp_m, tmp_g_m))]
    gr = tmp_g[tf.math.logical_not(tf.math.logical_and(tmp_m, tmp_g_m))]
    
    if pr is not None:
        c = tf.math.reduce_sum(tf.cast(pr==gr, dtype=tf.int64))/tf.cast(tf.size(pr), dtype=tf.int64)
        return c
    else:
        return 0.0

def go_val_f1(y_true, y_pred):
    tmp_p = tf.math.argmax(y_pred[0], axis=-1)
    tmp_g = tf.math.argmax(y_true[0], axis=-1)
    #mask 0
    tmp_p_m = tf.math.equal(tmp_p, tf.constant(0, dtype=tf.int64))
    tmp_g_m = tf.math.equal(tmp_g, tf.constant(0, dtype=tf.int64))
    
    #get the positions where 0 != 0
    pr = y_pred[tf.math.logical_not(tf.math.logical_and(tmp_p_m, tmp_g_m))]
    gr = y_true[tf.math.logical_not(tf.math.logical_and(tmp_p_m, tmp_g_m))]
    
    true_positives = K.sum(K.round(K.clip(gr * pr, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(gr, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(pr, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
def get_optimizer(opt, lr):
    if opt == 'adam':
        return tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    elif opt == 'adamw':
        step = tf.Variable(0, trainable=False)
        schedule = tf.compat.v1.keras.optimizers.schedules.PiecewiseConstantDecay(
        # schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
            [10000, 15000], [1e-0, 1e-1, 1e-2])
        # lr and wd can be a function or a tensor
        lr = lr * schedule(step)
        wd = lambda: 1e-4 * schedule(step)
        return tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
    elif opt == 'rmsprop':
        return tf.keras.optimizers.RMSprop(learning_rate=lr)
    
def get_loss(loss):
    if 'categoricalCE' in loss:
        if 'logits' in loss:
            return tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        else:
            return tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    elif 'sigfocalCE' in loss:
        if 'logits' in loss:
            return tfa.losses.SigmoidFocalCrossEntropy(from_logits=True),
        else:
            return tfa.losses.SigmoidFocalCrossEntropy(from_logits=False),
    elif 'custom_sigfocalCE' in loss:
        if 'logits' in loss:
            return SigFocalCrossEntropy(from_logits=True),
        else:
            return SigFocalCrossEntropy(from_logits=False)

In [ ]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32) #tf.constant(BATCH_SIZE*[MAX_WORD_LEN])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]


sess = tf.Session()
set_session(sess)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
word_in = Input(shape=(config.get("max_len"),), dtype=tf.string, name="WORD")
emb_word = Lambda(ElmoEmbedding)(word_in)

# input and embeddings for characters
char_in = Input(shape=(config.get("max_len"), config.get("max_char_len"),), name="CHAR")
emb_char = TimeDistributed(Embedding(input_dim=n_chars, output_dim=100, input_length=config.get("max_char_len"), mask_zero=True))(char_in)
char_enc = TimeDistributed(GRU(units=150, return_sequences=False, recurrent_dropout=0.5))(emb_char)

#input and embeddings for pos
pos_in = Input(shape=(config.get("max_len"),), name="POS")
emb_pos = Embedding(input_dim=n_pos, output_dim=100, input_length=config.get("max_len"), mask_zero=True, name="EMB_POS")(pos_in)

# main LSTM
x = concatenate([emb_word, char_enc, emb_pos])
x = SpatialDropout1D(config.get("dropout"))(x)
main_lstm = Bidirectional(GRU(units=150, return_sequences=True, recurrent_dropout=config['rdropout']))(x)
main_lstm = TimeDistributed(Dense(3200, activation='relu'))(main_lstm)
main_lstm = Dropout(config.get("dropout"))(main_lstm)
out = TimeDistributed(Dense(n_tags, activation=config['activation']), name="OUT_TAGS")(main_lstm)

model = Model([word_in, char_in, pos_in], out)
model.compile(optimizer = get_optimizer(config['optimizer'], config['learning_rate']), loss=get_loss(config['loss']), metrics=["acc", go_val_f1])

In [ ]:
model.summary()

In [ ]:
model_arch = tf.keras.utils.plot_model(model, show_shapes=True)
display(model_arch)

In [ ]:
sess = tf.Session()
set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
history = model.fit(
    input_train, y_tr, 
    batch_size=config.get("batch_size"),
    epochs=config.get("epochs"),
    verbose=1,
)

In [ ]:
del X_word, X_char, X_pos, Y_tags, combined

In [ ]:
pred = []
pbar = tqdm(total=len(X_te), desc="Making predictions:")
for i in range(int(len(X_te)/10)):
    # temp = np.argsort(-1*model.predict([j[i*10:(i+1)*10] for j in input_test]), axis=-1)
    inp = [j[i*10:(i+1)*10] for j in input_test]
    if inp[0].shape[0] != 0:
        temp = model.predict(inp)
    pred.append(temp)
    pbar.update(temp.shape[0])
pred = np.concatenate(pred)
pbar.close()
pred.shape

In [ ]:
word = input_test[0].flatten()
ground_truth = np.vectorize(idx2tag.get)(np.argmax(y_te, axis=-1)).flatten()
predictions = (np.vectorize(idx2tag.get)(np.argmax(pred, axis=-1))).flatten().tolist()
two_predictions = (np.vectorize(idx2tag.get)(np.argsort(-1*pred, axis=-1)[:,:,:2])).reshape(pred.shape[0]*pred.shape[1], 2).tolist()

In [ ]:
pd_data = pd.DataFrame({
    "Word": word,
    "Ground_Truth": ground_truth,
    "Prediction": predictions,
    "Top_Two_Predictions": two_predictions,
})
pd_data

In [ ]:
pd_data.drop(pd_data[pd_data['Ground_Truth'] == "PAD"].index, inplace=True)
pd_data.drop(pd_data[(pd_data["Ground_Truth"] == "O") & (pd_data["Prediction"] == "O")].index, inplace=True)
pd_data.drop(pd_data[(pd_data["Ground_Truth"] == "EOS") & (pd_data["Prediction"] == "EOS")].index, inplace=True)
pd_data

In [ ]:
pd_data['Comparison'] = pd_data.apply(lambda x: x[1] if x[1] in x[-1] else x[-1][-1], axis=1)
pd_data

In [ ]:
top_report = classification_report(
    pd_data['Ground_Truth'],
    pd_data['Prediction'],
    zero_division=False,
    digits = 4
)
print(top_report)

In [ ]:
{"IOB_F1": top_report.splitlines()[-1].split()[-2],
                      "IOB_Sim": np.round(pd_data[['Prediction','Ground_Truth']].apply(lambda x: get_sim(x[0], x[1]), axis=1).mean(), 4)}

In [ ]:
df1 = pd_data.copy().replace({"B-GO:": "GO:", "I-GO:": "GO:"}, regex=True)
df1

In [ ]:
report = classification_report(
    df1['Ground_Truth'],
    df1['Prediction'],
    zero_division=False,
    digits = 4
)
print(report)

In [ ]:
{"F1": report.splitlines()[-1].split()[-2],
                      "Sim": np.round(df1[['Prediction','Ground_Truth']].apply(lambda x: get_sim(x[0], x[1]), axis=1).mean(), 4)}

In [ ]:
df2 = df1.copy()
df2.drop(df2[(df2["Ground_Truth"] == "O") & (df2["Comparison"] == "O")].index, inplace=True)
df2.drop(df2[(df2["Ground_Truth"] == "EOS") & (df2["Comparison"] == "EOS")].index, inplace=True)
df2

In [ ]:
top_two_report = classification_report(
    df2['Ground_Truth'],
    df2['Comparison'],
    zero_division=False,
    digits = 4
)
print(top_two_report)

In [ ]:
{"F1_Top_2": top_two_report.splitlines()[-1].split()[-2],
                      "Sim_Top_2": np.round(df2[['Comparison','Ground_Truth']].apply(lambda x: get_sim(x[0], x[1]), axis=1).mean(), 4)}